# ⚙️ Configuration de l’environnement Python & validation du token Hugging Face

Bienvenue !  
Ce notebook vérifie que :
- Ton environnement technique est prêt.
- Ton token Hugging Face est sécurisé et fonctionnel.
- L’accès à l’API serverless de segmentation est possible.

Chaque étape affiche une coche verte ou une croix rouge selon le résultat.

In [4]:
# 1. Version de Python et répertoire d’exécution
import sys
import os
print(f"Version Python : {sys.version}")
print(f"Répertoire courant : {os.getcwd()}")

Version Python : 3.11.8 (main, Jul 17 2025, 14:26:09) [Clang 17.0.0 (clang-1700.0.13.5)]
Répertoire courant : /Users/daminou/Documents/Formation_IA/Projet_2/fashion-trend-intelligence/notebooks


In [5]:
# 2. Vérification de l’environnement virtuel Poetry
venv = os.environ.get('VIRTUAL_ENV')
ok_env = bool(venv)
if ok_env:
    print(f"✅ Environnement Poetry actif ({venv})")
else:
    print("❌ Poetry n'est pas activé : pense à lancer 'poetry shell' dans le terminal.")

✅ Environnement Poetry actif (/Users/daminou/Documents/Formation_IA/Projet_2/fashion-trend-intelligence/.venv)


In [6]:
# 3. Dépendances principales installées
ok_dependencies = True
for pkg in ['requests', 'dotenv', 'numpy', 'matplotlib', 'PIL', 'huggingface_hub']:
    try:
        __import__(pkg)
    except ImportError:
        ok_dependencies = False
        print(f"❌ Package manquant : {pkg}")
if ok_dependencies:
    print("✅ Toutes les dépendances principales sont installées.")

✅ Toutes les dépendances principales sont installées.


In [7]:
# 4. Présence et chargement du fichier .env (où se trouve le token)
from pathlib import Path
from dotenv import load_dotenv

def locate_dotenv():
    path = Path.cwd()
    while path != path.parent:
        candidate = path / ".env"
        if candidate.exists():
            return candidate
        path = path.parent
    raise FileNotFoundError(".env introuvable dans les dossiers parents")

try:
    dotenv_path = locate_dotenv()
    ok_envfile = True
    print(f"✅ Fichier .env trouvé dans : {dotenv_path}")
except Exception as e:
    ok_envfile = False
    print(f"❌ {e}")

ok_token = False
import os
if ok_envfile:
    load_dotenv(dotenv_path=dotenv_path)
    HF_TOKEN = os.environ.get("HF_TOKEN")
    if HF_TOKEN and HF_TOKEN.startswith("hf_"):
        ok_token = True
        print("✅ Token Hugging Face chargé et au bon format.")
    else:
        print("❌ Token Hugging Face absent ou mal formaté dans .env.")

✅ Fichier .env trouvé dans : /Users/daminou/Documents/Formation_IA/Projet_2/fashion-trend-intelligence/.env
✅ Token Hugging Face chargé et au bon format.


In [8]:
# 5. Test d’appel API Hugging Face
ok_api_call = False
if ok_token:
    import requests
    model_id = "sayeed99/segformer_b3_clothes"
    url = f"https://api-inference.huggingface.co/models/{model_id}"
    try:
        resp = requests.get(url, headers={"Authorization": f"Bearer {HF_TOKEN}"}, timeout=10)
        if resp.status_code == 200:
            ok_api_call = True
            print("✅ L'API Hugging Face et le token fonctionnent !")
        elif resp.status_code in (401, 403):
            print("❌ Mauvais token ou permissions insuffisantes.")
        else:
            print(f"⚠️ Code retour inattendu ({resp.status_code}) : {resp.text[:100]}…")
    except Exception as e:
        print(f"❌ Erreur lors du test API : {e}")
else:
    print("⏭️  Test API sauté puisque le token n’est pas valide.")

✅ L'API Hugging Face et le token fonctionnent !


In [10]:
# 6. Rapport global de validation – tout en un clin d’œil
from IPython.display import display, Markdown

tests = [
    ("Présence du .env", ok_envfile),
    ("Token Hugging Face chargé", ok_token),
    ("Dépendances principales installées", ok_dependencies),
    ("Test API Hugging Face (statut 200)", ok_api_call),
]

rapport_md = "### État du setup<br/>\n\n"
for label, status in tests:
    icon = "✅" if status else "❌"
    color = "green" if status else "red"
    rapport_md += f"- <span style='color:{color}; font-weight:bold'>{icon} {label}</span><br/>\n"

all_ok = all(status for _, status in tests)
if all_ok:
    rapport_md += "\n---\n<span style='color:green; font-weight:bold; font-size:1.1em;'>✅ Tout est prêt ! 🎉</span>"
else:
    rapport_md += "\n---\n<span style='color:red; font-weight:bold; font-size:1.1em;'>❌ Au moins une étape clé a échoué. Voir en rouge ci-dessus.</span>"

display(Markdown(rapport_md))


### État du setup<br/>

- <span style='color:green; font-weight:bold'>✅ Présence du .env</span><br/>
- <span style='color:green; font-weight:bold'>✅ Token Hugging Face chargé</span><br/>
- <span style='color:green; font-weight:bold'>✅ Dépendances principales installées</span><br/>
- <span style='color:green; font-weight:bold'>✅ Test API Hugging Face (statut 200)</span><br/>

---
<span style='color:green; font-weight:bold; font-size:1.1em;'>✅ Tout est prêt ! 🎉</span>